In [64]:
!pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [1]:
!pip install tensorflow-gpu==2.8.0


In [2]:
pip install plotly

In [4]:
import tensorflow as tf

# Check if GPU is available
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "not available")

# Perform a simple computation on GPU
with tf.device('/GPU:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0])
    b = tf.constant([5.0, 4.0, 3.0, 2.0, 1.0])
    c = a + b

# Print the result
print(c)


GPU is not available
tf.Tensor([6. 6. 6. 6. 6.], shape=(5,), dtype=float32)


# Import Libraries

In [1]:
import os
import random
import shutil
import zipfile
import operator
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Add
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D

In [2]:
print("TensorFlow version:", tf.__version__)

# Check if GPU is available
if tf.test.gpu_device_name():
    print('GPU:', tf.test.gpu_device_name())
else:
    print("No GPU available")

# Check if TPU is available
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("TPU:", resolver.master())
except Exception as e:
    print("No TPU available:", e)


TensorFlow version: 2.15.0
GPU: /device:GPU:0
No TPU available: Please provide a TPU Name to connect to.


# Normalize and Visualize Dataset

In [4]:
# Function to count files in each folder
def count_files_in_folders(root_dir):
    folder_counts = {}
    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)
        if os.path.isdir(folder_path):
            folder_counts[folder] = len(os.listdir(folder_path))
    return folder_counts

# Function to normalize images
def normalize_images(image_paths):
    images = []
    for img_path in image_paths:
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        normalized_img = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        images.append(normalized_img)
    return images

# Function to unzip a folder
def unzip_folder(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Path to your zipped folder
zip_file = "/content/gwbz3fsgp8-1.zip"

# Directory to extract the contents of the zip file
extract_to = "path/to/extracted/folder"

# Create the directory if it doesn't exist
if not os.path.exists(extract_to):
    os.makedirs(extract_to)

# Unzip the folder
unzip_folder(zip_file, extract_to)

# Path to the content folder containing all five image folders
content_folder = extract_to

# Count images in each folder
folder_counts = count_files_in_folders(content_folder)

# Create figure
fig = go.Figure()

# Define colors for bars
colors = ['lightskyblue', 'lightgreen', 'salmon', 'lightyellow', 'lightpink']

for i, folder in enumerate(folder_counts.keys()):
    fig.add_trace(go.Bar(
        x=[folder],
        y=[folder_counts[folder]],
        text=f"{folder}: {folder_counts[folder]} images",
        hoverinfo="text",
        textposition='auto',
        marker_color=colors[i % len(colors)]  # Use modulo operator to cycle through colors
    ))

# Update layout
fig.update_layout(
    title='Distribution of Images in Folders',
    title_x=0.5,
    xaxis=dict(title='Data Folders', title_standoff=25, showticklabels=False),
    yaxis=dict(title='Number of Images'),
    showlegend=False
)

# Show plot
fig.show()

# Normalize images
normalized_images = {}
for folder in folder_counts.keys():
    folder_path = os.path.join(content_folder, folder)
    image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path)]
    normalized_images[folder] = normalize_images(image_paths)

# Now normalized_images contains normalized images for each folder

In [5]:
# Sort folder_counts by counts
sorted_folder_counts = dict(sorted(folder_counts.items(), key=operator.itemgetter(1), reverse=True))

# Extract top two folders
top_folders = list(sorted_folder_counts.keys())[:2]

# Define colors for bars
top_color = '#bd3c3c'
other_color = 'lightgrey'

# Create figure
fig = go.Figure()

for i, folder in enumerate(sorted_folder_counts.keys()):
    bar_color = top_color if folder in top_folders else other_color
    border_color = 'red' if folder in top_folders else None
    fig.add_trace(go.Bar(
        x=[folder],
        y=[sorted_folder_counts[folder]],
        text=f"{folder}: {sorted_folder_counts[folder]} images",
        hoverinfo="text",
        textposition='auto',
        marker_color=bar_color,
        marker_line_color=border_color,
        marker_line_width=1.5
    ))

# Update layout
fig.update_layout(
    title='Distribution of Images in Folders',
    title_x=0.5,
    xaxis=dict(title='Data Folders', title_standoff=25, showticklabels=False),
    yaxis=dict(title='Number of Images'),
    showlegend=False,
    plot_bgcolor='#13141a',
    paper_bgcolor='#13141a',
    font=dict(color='lightgray')
)

# Show plot
fig.show()


# Filtering the Dataset - Normal and Abnormal Hear Beat (540 images each)

In [7]:
# Function to randomly select images from a folder
def random_select_images(folder_path, num_images):
    image_files = os.listdir(folder_path)
    selected_images = random.sample(image_files, num_images)
    return selected_images

# Define paths to the "ECG - Normal" and "ECG - Abnormal" folders
normal_folder = "/content/path/to/extracted/folder/Normal Person ECG Images (859)"
abnormal_folder = "/content/path/to/extracted/folder/ECG Images of Patient that have abnormal heart beats (548)"

# Define paths for the new folders that will contain selected images
selected_normal_folder = "/content/selected_normal"
selected_abnormal_folder = "/content/selected_abnormal"

# Create directories for the selected folders
os.makedirs(selected_normal_folder, exist_ok=True)
os.makedirs(selected_abnormal_folder, exist_ok=True)

# Randomly select and move 540 images from each folder to the new folders
num_images_per_set = 540
selected_normal_images = random_select_images(normal_folder, num_images_per_set)
selected_abnormal_images = random_select_images(abnormal_folder, num_images_per_set)

for image in selected_normal_images:
    src_path = os.path.join(normal_folder, image)
    dest_path = os.path.join(selected_normal_folder, image)
    shutil.copy(src_path, dest_path)

for image in selected_abnormal_images:
    src_path = os.path.join(abnormal_folder, image)
    dest_path = os.path.join(selected_abnormal_folder, image)
    shutil.copy(src_path, dest_path)

# Print the counts
print("Number of images selected for normal folder:", len(selected_normal_images))
print("Number of images selected for abnormal folder:", len(selected_abnormal_images))


Number of images selected for normal folder: 540
Number of images selected for abnormal folder: 540


# Splitting the Dataset into Training, Testing and Validation (80-10-10)

In [8]:
# Function to randomly select images from a folder
def random_select_images(folder_path, num_images):
    image_files = os.listdir(folder_path)
    selected_images = random.sample(image_files, num_images)
    return selected_images

# Define paths for the new folders that will contain selected images
selected_normal_folder = "/content/selected_normal"
selected_abnormal_folder = "/content/selected_abnormal"

# Define paths for the new folders that will contain training, testing, and validation sets
train_normal_folder = "/content/train/normal"
train_abnormal_folder = "/content/train/abnormal"
test_normal_folder = "/content/test/normal"
test_abnormal_folder = "/content/test/abnormal"
val_normal_folder = "/content/val/normal"
val_abnormal_folder = "/content/val/abnormal"

# Create directories for the new folders
for folder in [train_normal_folder, train_abnormal_folder, test_normal_folder, test_abnormal_folder, val_normal_folder, val_abnormal_folder]:
    os.makedirs(folder, exist_ok=True)

# Split selected images into training, testing, and validation sets
normal_images = os.listdir(selected_normal_folder)
abnormal_images = os.listdir(selected_abnormal_folder)

# Splitting the normal images
train_normal, testval_normal = train_test_split(normal_images, test_size=0.3, random_state=42)
test_normal, val_normal = train_test_split(testval_normal, test_size=0.5, random_state=42)

# Splitting the abnormal images
train_abnormal, testval_abnormal = train_test_split(abnormal_images, test_size=0.3, random_state=42)
test_abnormal, val_abnormal = train_test_split(testval_abnormal, test_size=0.5, random_state=42)

# Move images to respective folders
for image in train_normal:
    src_path = os.path.join(selected_normal_folder, image)
    dest_path = os.path.join(train_normal_folder, image)
    shutil.copy(src_path, dest_path)

for image in test_normal:
    src_path = os.path.join(selected_normal_folder, image)
    dest_path = os.path.join(test_normal_folder, image)
    shutil.copy(src_path, dest_path)

for image in val_normal:
    src_path = os.path.join(selected_normal_folder, image)
    dest_path = os.path.join(val_normal_folder, image)
    shutil.copy(src_path, dest_path)

for image in train_abnormal:
    src_path = os.path.join(selected_abnormal_folder, image)
    dest_path = os.path.join(train_abnormal_folder, image)
    shutil.copy(src_path, dest_path)

for image in test_abnormal:
    src_path = os.path.join(selected_abnormal_folder, image)
    dest_path = os.path.join(test_abnormal_folder, image)
    shutil.copy(src_path, dest_path)

for image in val_abnormal:
    src_path = os.path.join(selected_abnormal_folder, image)
    dest_path = os.path.join(val_abnormal_folder, image)
    shutil.copy(src_path, dest_path)

# Print counts
print("Training set size (normal):", len(train_normal))
print("Testing set size (normal):", len(test_normal))
print("Validation set size (normal):", len(val_normal))
print("Training set size (abnormal):", len(train_abnormal))
print("Testing set size (abnormal):", len(test_abnormal))
print("Validation set size (abnormal):", len(val_abnormal))

Training set size (normal): 378
Testing set size (normal): 81
Validation set size (normal): 81
Training set size (abnormal): 378
Testing set size (abnormal): 81
Validation set size (abnormal): 81


# Create a CNN Model

In [10]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define data directories
train_dir = "/content/train"
val_dir = "/content/val"
test_dir = "/content/test"

# Define image dimensions
img_width, img_height = 224, 224

# Define batch size
batch_size = 32

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

# Prepare data generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode='binary')

test_generator = val_datagen.flow_from_directory(test_dir,
                                                 target_size=(img_width, img_height),
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // batch_size)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy:", test_accuracy)


Found 756 images belonging to 2 classes.
Found 162 images belonging to 2 classes.
Found 162 images belonging to 2 classes.
Epoch 1/100
23/23 [==============================] - 30s 1s/step - loss: 0.7381 - accuracy: 0.5097 - val_loss: 0.6889 - val_accuracy: 0.5000
Epoch 2/100
23/23 [==============================] - 27s 1s/step - loss: 0.6774 - accuracy: 0.5677 - val_loss: 0.6483 - val_accuracy: 0.4938
Epoch 3/100
23/23 [==============================] - 27s 1s/step - loss: 0.6466 - accuracy: 0.5994 - val_loss: 1.0068 - val_accuracy: 0.5188
Epoch 4/100
23/23 [==============================] - 27s 1s/step - loss: 0.5704 - accuracy: 0.6948 - val_loss: 0.8938 - val_accuracy: 0.5375
Epoch 5/100
23/23 [==============================] - 27s 1s/step - loss: 0.5064 - accuracy: 0.7500 - val_loss: 0.5786 - val_accuracy: 0.7563
Epoch 6/100
23/23 [==============================] - 27s 1s/step - loss: 0.4562 - accuracy: 0.7859 - val_loss: 1.0188 - val_accuracy: 0.5312
Epoch 7/100
23/23 [============

KeyboardInterrupt: 

# ResNet50 Model

In [11]:
# Load the pre-trained ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of ResNet50
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // batch_size)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy:", test_accuracy)

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/20
23/23 [==============================] - 33s 1s/step - loss: 0.7140 - accuracy: 0.5476 - val_loss: 0.6118 - val_accuracy: 0.5875
Epoch 2/20
23/23 [==============================] - 28s 1s/step - loss: 0.5828 - accuracy: 0.7072 - val_loss: 0.6098 - val_accuracy: 0.6000
Epoch 3/20
23/23 [==============================] - 26s 1s/step - loss: 0.5587 - accuracy: 0.6851 - val_loss: 0.6143 - val_accuracy: 0.6187
Epoch 4/20
23/23 [==============================] - 26s 1s/step - loss: 0.5377 - accuracy: 0.7238 - val_loss: 0.7082 - val_accuracy: 0.5375
Epoch 5/20
23/23 [==============================] - 27s 1s/step - loss: 0.5268 - accuracy: 0.7086 - val_loss: 0.5554 - val_accuracy: 0.7000
Epoch 6/20
23/23 [==============================] - 26s 1s/step - loss: 0.5165 - accuracy: 0.7293 - val_loss: 0.6009 - val_accuracy: 0.6625
Epoch 7/20
23/23 [==============================] - 26s 1s/step - loss: 0.5318 - accuracy: 0.72

In [20]:
# Extracting training and validation metrics from history object
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Extracting epochs
epochs_list = list(range(1, len(train_accuracy) + 1))

# Define colors for training and validation lines
train_color = 'red'
val_color = 'blue'

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Accuracy", "Loss"))

# Add traces for accuracy
fig.add_trace(go.Scatter(x=epochs_list, y=train_accuracy, mode='lines', name='Training accuracy', line=dict(color=train_color)), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs_list, y=val_accuracy, mode='lines', name='Validation accuracy', line=dict(color=val_color)), row=1, col=1)

# Add traces for loss
fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training loss', line=dict(color=train_color)), row=1, col=2)
fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation loss', line=dict(color=val_color)), row=1, col=2)

# Update layout
fig.update_layout(title_text="ResNet50 Training and Validation Metrics Over Epochs", title_x=0.5)
fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_xaxes(title_text="Epochs", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=2)

# Show plot
fig.show()


In [22]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Unfreeze some top layers for fine-tuning
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Add regularization (e.g., dropout) to custom classification layers
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),  # Add dropout layer
    Dense(1, activation='sigmoid')
])

# Implement early stopping and learning rate reduction
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=2)

# Compile the model with a lower initial learning rate
model.compile(optimizer=Adam(lr=0.001),  # Lower learning rate
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with augmented data
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=20,  # Increase epochs
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // batch_size,
                    callbacks=[early_stopping, reduce_lr])  # Add callbacks for early stopping and reducing LR


Epoch 1/20
23/23 [==============================] - 63s 1s/step - loss: 0.6234 - accuracy: 0.7113 - val_loss: 14227.4004 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/20
23/23 [==============================] - 32s 1s/step - loss: 0.3938 - accuracy: 0.8232 - val_loss: 17313.8398 - val_accuracy: 0.4938 - lr: 0.0010
Epoch 3/20
23/23 [==============================] - 28s 1s/step - loss: 0.3588 - accuracy: 0.8508 - val_loss: 97.4371 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/20
23/23 [==============================] - 29s 1s/step - loss: 0.3532 - accuracy: 0.8425 - val_loss: 0.7302 - val_accuracy: 0.5063 - lr: 0.0010
Epoch 5/20
23/23 [==============================] - 30s 1s/step - loss: 0.3369 - accuracy: 0.8508 - val_loss: 0.8140 - val_accuracy: 0.5063 - lr: 0.0010
Epoch 6/20
23/23 [==============================] - 28s 1s/step - loss: 0.3462 - accuracy: 0.8536 - val_loss: 0.7137 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 7/20
23/23 [==============================] - 28s 1s/step - loss: 0

In [30]:
# Extracting training and validation metrics from history object
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Extracting epochs
epochs_list = list(range(1, len(train_accuracy) + 1))

# Define colors for training and validation lines
train_color = 'red'
val_color = 'blue'

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Accuracy", "Loss"))

# Add traces for accuracy
fig.add_trace(go.Scatter(x=epochs_list, y=train_accuracy, mode='lines', name='Training accuracy', line=dict(color=train_color)), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs_list, y=val_accuracy, mode='lines', name='Validation accuracy', line=dict(color=val_color)), row=1, col=1)

# Add traces for loss
fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training loss', line=dict(color=train_color)), row=1, col=2)
fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation loss', line=dict(color=val_color)), row=1, col=2)

# Update layout
fig.update_layout(title_text="ResNet50 Training and Validation Metrics Over Epochs (wtih early stopping and reducing learning rate)", title_x=0.5)
fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_xaxes(title_text="Epochs", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=2)

# Show plot
fig.show()

# ResNet100 Model


In [47]:
from tensorflow.keras.applications import ResNet101

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define data directories
train_dir = "/content/train"
val_dir = "/content/val"
test_dir = "/content/test"

# Define image dimensions
img_width, img_height = 224, 224

# Define batch size
batch_size = 32

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

# Prepare data generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode='binary')

test_generator = val_datagen.flow_from_directory(test_dir,
                                                 target_size=(img_width, img_height),
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Load the pre-trained ResNet101 model without the top classification layer
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of ResNet101
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=20,
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // batch_size)

# Evaluate the model on the test data
test_loss, test_accuracy100 = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy:", test_accuracy100)


Found 756 images belonging to 2 classes.
Found 162 images belonging to 2 classes.
Found 162 images belonging to 2 classes.
Epoch 1/20
23/23 [==============================] - 46s 2s/step - loss: 0.6651 - accuracy: 0.5870 - val_loss: 0.5979 - val_accuracy: 0.6687
Epoch 2/20
23/23 [==============================] - 32s 1s/step - loss: 0.5747 - accuracy: 0.6906 - val_loss: 0.5518 - val_accuracy: 0.6750
Epoch 3/20
23/23 [==============================] - 31s 1s/step - loss: 0.5408 - accuracy: 0.7127 - val_loss: 0.5493 - val_accuracy: 0.7500
Epoch 4/20
23/23 [==============================] - 28s 1s/step - loss: 0.5402 - accuracy: 0.7196 - val_loss: 0.5121 - val_accuracy: 0.7500
Epoch 5/20
23/23 [==============================] - 29s 1s/step - loss: 0.5148 - accuracy: 0.7417 - val_loss: 0.5464 - val_accuracy: 0.7437
Epoch 6/20
23/23 [==============================] - 32s 1s/step - loss: 0.5085 - accuracy: 0.7486 - val_loss: 0.4952 - val_accuracy: 0.7437
Epoch 7/20
23/23 [===================

In [50]:
# Extracting training and validation metrics from history object
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Extracting epochs
epochs_list = list(range(1, len(train_accuracy) + 1))

# Define colors for training and validation lines
train_color = 'red'
val_color = 'blue'

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Accuracy", "Loss"))

# Add traces for accuracy
fig.add_trace(go.Scatter(x=epochs_list, y=train_accuracy, mode='lines', name='Training accuracy', line=dict(color=train_color)), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs_list, y=val_accuracy, mode='lines', name='Validation accuracy', line=dict(color=val_color)), row=1, col=1)

# Add traces for loss
fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training loss', line=dict(color=train_color)), row=1, col=2)
fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation loss', line=dict(color=val_color)), row=1, col=2)

# Update layout
fig.update_layout(title_text="ResNet100 Training and Validation Metrics Over Epochs", title_x=0.5)
fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_xaxes(title_text="Epochs", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=2)

# Show plot
fig.show()


# Aleksandra's ResNet Model

In [53]:
def residual_block(x, filters, strides=1, activation='relu'):
    shortcut = x
    x = Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    if strides != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    x = Add()([x, shortcut])
    x = Activation(activation)(x)
    return x

def custom_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=(7, 7), strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=2)
    x = residual_block(x, filters=128)
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Define input shape and number of classes
input_shape = (224, 224, 2)
num_classes = 2  # Binary classification (normal vs. abnormal)

# Create the custom ResNet model
model = custom_resnet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 224, 224, 2)]        0         []                            
                                                                                                  
 conv2d_40 (Conv2D)          (None, 112, 112, 64)         6336      ['input_10[0][0]']            
                                                                                                  
 batch_normalization_20 (Ba  (None, 112, 112, 64)         256       ['conv2d_40[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 112, 112, 64)         0         ['batch_normalization_20

In [ ]:
# Define paths for the new folders that will contain training, testing, and validation sets
train_normal_folder = "/content/train/normal"
train_abnormal_folder = "/content/train/abnormal"
test_normal_folder = "/content/test/normal"
test_abnormal_folder = "/content/test/abnormal"
val_normal_folder = "/content/val/normal"
val_abnormal_folder = "/content/val/abnormal"

# Define image dimensions
img_width, img_height = 224, 224

# Define batch size
batch_size = 32

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

# Prepare data generators
train_generator = train_datagen.flow_from_directory("/content/train",
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory("/content/val",
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode='binary')

test_generator = val_datagen.flow_from_directory("/content/test",
                                                 target_size=(img_width, img_height),
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=50,
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // batch_size)

# Evaluate the model on the test data
test_loss, test_accuracy_mine = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy:", test_accuracy_mine)


Found 756 images belonging to 2 classes.
Found 162 images belonging to 2 classes.
Found 162 images belonging to 2 classes.
Epoch 1/50
23/23 [==============================] - 30s 1s/step - loss: 0.7546 - accuracy: 0.5152 - val_loss: 0.7129 - val_accuracy: 0.5000
Epoch 2/50
23/23 [==============================] - 29s 1s/step - loss: 0.6954 - accuracy: 0.4945 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/50
23/23 [==============================] - 27s 1s/step - loss: 0.6932 - accuracy: 0.5028 - val_loss: 0.6931 - val_accuracy: 0.5063
Epoch 4/50
23/23 [==============================] - 26s 1s/step - loss: 0.6932 - accuracy: 0.4862 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/50
23/23 [==============================] - 29s 1s/step - loss: 0.6933 - accuracy: 0.4903 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 6/50
23/23 [==============================] - 33s 1s/step - loss: 0.6932 - accuracy: 0.4862 - val_loss: 0.6839 - val_accuracy: 0.6313
Epoch 7/50
23/23 [===================

In [34]:
# Extracting training and validation metrics from history object
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Extracting epochs
epochs_list = list(range(1, len(train_accuracy) + 1))

# Define colors for training and validation lines
train_color = 'red'
val_color = 'blue'

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Accuracy", "Loss"))

# Add traces for accuracy
fig.add_trace(go.Scatter(x=epochs_list, y=train_accuracy, mode='lines', name='Training accuracy', line=dict(color=train_color)), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs_list, y=val_accuracy, mode='lines', name='Validation accuracy', line=dict(color=val_color)), row=1, col=1)

# Add traces for loss
fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training loss', line=dict(color=train_color)), row=1, col=2)
fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation loss', line=dict(color=val_color)), row=1, col=2)

# Update layout
fig.update_layout(title_text="Aleksandra's ResNet Model Training and Validation Metrics Over Epochs", title_x=0.5)
fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_xaxes(title_text="Epochs", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=2)

# Show plot
fig.show()


# VGG16 Model

In [40]:
from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model without the top classification layer
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model layers
for layer in base_model_vgg16.layers:
    layer.trainable = False

# Add custom classification layers on top of VGG16
model_vgg16 = Sequential([
    base_model_vgg16,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model_vgg16.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

# Train the model
history_vgg16 = model_vgg16.fit(train_generator,
                                steps_per_epoch=train_generator.samples // batch_size,
                                epochs=20,
                                validation_data=val_generator,
                                validation_steps=val_generator.samples // batch_size)

# Evaluate the model on the test data
test_loss_vgg16, test_accuracy_vgg16 = model_vgg16.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy (VGG16):", test_accuracy_vgg16)


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/20
23/23 [==============================] - 36s 1s/step - loss: 1.1893 - accuracy: 0.6492 - val_loss: 0.7287 - val_accuracy: 0.5938
Epoch 2/20
23/23 [==============================] - 26s 1s/step - loss: 0.4430 - accuracy: 0.7970 - val_loss: 0.4375 - val_accuracy: 0.8000
Epoch 3/20
23/23 [==============================] - 26s 1s/step - loss: 0.4050 - accuracy: 0.8191 - val_loss: 0.4244 - val_accuracy: 0.7750
Epoch 4/20
23/23 [==============================] - 27s 1s/step - loss: 0.3903 - accuracy: 0.8163 - val_loss: 0.4183 - val_accuracy: 0.8000
Epoch 5/20
23/23 [==============================] - 26s 1s/step - loss: 0.4081 - accuracy: 0.8149 - val_loss: 0.7425 - val_accuracy: 0.6125
Epoch 6/20
23/23 [==============================] - 25s 1s/step - loss: 0.4108 - accuracy: 0.8149 - val_loss: 0.5616 - val_accuracy: 0.7437
Epoch 7/20
23/23 [==============================] - 26s 1s/step - loss: 0.4341 - accuracy: 0.81

In [41]:
# Extracting training and validation metrics from history object
train_accuracy_vgg16 = history_vgg16.history['accuracy']
val_accuracy_vgg16 = history_vgg16.history['val_accuracy']
train_loss_vgg16 = history_vgg16.history['loss']
val_loss_vgg16 = history_vgg16.history['val_loss']

# Extracting epochs
epochs_list_vgg16 = list(range(1, len(train_accuracy_vgg16) + 1))

# Define colors for training and validation lines
train_color = 'red'
val_color = 'blue'

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Accuracy (VGG16)", "Loss (VGG16)"))

# Add traces for accuracy
fig.add_trace(go.Scatter(x=epochs_list_vgg16, y=train_accuracy_vgg16, mode='lines', name='Training accuracy', line=dict(color=train_color)), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs_list_vgg16, y=val_accuracy_vgg16, mode='lines', name='Validation accuracy', line=dict(color=val_color)), row=1, col=1)

# Add traces for loss
fig.add_trace(go.Scatter(x=epochs_list_vgg16, y=train_loss_vgg16, mode='lines', name='Training loss', line=dict(color=train_color)), row=1, col=2)
fig.add_trace(go.Scatter(x=epochs_list_vgg16, y=val_loss_vgg16, mode='lines', name='Validation loss', line=dict(color=val_color)), row=1, col=2)

# Update layout
fig.update_layout(title_text="VGG16 Training and Validation Metrics Over Epochs", title_x=0.5)
fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_xaxes(title_text="Epochs", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=2)

# Show plot
fig.show()


In [52]:
# Define model names
model_names = ["ResNet50", "ResNet100", "ResNet", "VGG16"]

# Define test accuracies
test_accuracies = [test_accuracy, test_accuracy100, test_accuracy_mine, test_accuracy_vgg16]

# Create bar plot
fig = go.Figure(data=[go.Bar(x=model_names, y=test_accuracies, marker_color='skyblue')])
fig.update_layout(title='Comparison of Test Accuracies for Different Models',
                  xaxis_title='Model',
                  yaxis_title='Test Accuracy',
                  xaxis_tickangle=-45)
fig.show()